This is a simple web scraping code to get data about neighborhoods in Toronto from a Wikipedia html table (postal code, burough and neighborhood name) into a Pandas dataframe. The most interesting part of the code collapses the dataframe to aggregate all neighborhood names within the same Burough separating using commmas.

Rules of the game:
The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.

I only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re


url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_st=requests.get(url)
soup  = BeautifulSoup(html_st.text,'html.parser')
soup1 = soup.table
soup2 = soup1.find('tbody')
soup3 = soup2.find_all('tr')

data=[]
varnames=[]
first=1
for row in soup3:
 cols = row.find_all('td')
 cols = [ele.text.strip() for ele in cols]
 if len(cols)==0:
  varnames  = row.find_all('th')
  varnames = [ele.text.strip() for ele in varnames]
  #varnames.append([ele for ele in cols if ele])
 else:
  if cols[1]!='Not assigned' and cols[2]=='Not assigned':  
   cols[2]=cols[1]
  if cols[0]!='None' and cols[1]!='Not assigned':
   if first==1:
    data=[cols]
    first=0
   else:
    data.append([ele for ele in cols if ele])
df1 = pd.DataFrame(data,columns=varnames)
df=df1.sort_values(by=varnames[1:2])

print(varnames)
j=0
dfn=pd.DataFrame(columns=varnames)
acc=df.Neighbourhood[0]
for i in range(2,len(df)):
 if df.Borough[i]==df.Borough[i-1]:
  acc=acc+", "+df.Neighbourhood[i]
 else:
  aux1=df.Borough[i-1]
  aux2=df.Postcode[i-1]
  mini=pd.DataFrame.from_records([(aux2, aux1, acc)],columns=varnames)
  dfn=dfn.append(mini,ignore_index=True)
  j=j+1
  acc=df.Neighbourhood[i]
dfn.shape
#-------------------------------------------------------------------------------
urlgeo='https://cocl.us/Geospatial_data'
geodat=[]
s=requests.get(urlgeo).content
ss=re.split('\\\\r\\\\n',str(s))
for line in ss:
 linee=re.split(',',line)
 geodat.append(linee)

varn=geodat.pop(0)
print(varn)
print(varn[0])
varn[0]='Postcode'
geo_pd=pd.DataFrame(geodat,columns=varn)
df2=pd.merge(geo_pd,df)
print(df2)

['Postcode', 'Borough', 'Neighbourhood']
["b'Postal Code", 'Latitude', 'Longitude']
b'Postal Code
    Postcode    Latitude     Longitude      Borough             Neighbourhood
0        M1B  43.8066863   -79.1943534  Scarborough                   Malvern
1        M1B  43.8066863   -79.1943534  Scarborough                     Rouge
2        M1C  43.7845351   -79.1604971  Scarborough                Rouge Hill
3        M1C  43.7845351   -79.1604971  Scarborough                Port Union
4        M1C  43.7845351   -79.1604971  Scarborough            Highland Creek
5        M1E  43.7635726   -79.1887115  Scarborough                 West Hill
6        M1E  43.7635726   -79.1887115  Scarborough                 Guildwood
7        M1E  43.7635726   -79.1887115  Scarborough               Morningside
8        M1G  43.7709921   -79.2169174  Scarborough                    Woburn
9        M1H   43.773136   -79.2394761  Scarborough                 Cedarbrae
10       M1J  43.7447342   -79.2394761  Scar